In [40]:
# 📁 СОЗДАЕМ ФАЙЛ ДЛЯ ОБУЧЕНИЯ МОДЕЛИ
%%writefile train_titanic_model.py
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
import requests
from io import StringIO

# AI-ассистент: использован DeepSeek для генерации кода

print("🚀 Начинаем обучение модели Titanic...")

# Загрузка датасета Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

def preprocess_data(df):
    # Заполнение пропущенных значений
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)

    # Кодирование категориальных переменных
    le_sex = LabelEncoder()
    le_embarked = LabelEncoder()

    df['Sex_encoded'] = le_sex.fit_transform(df['Sex'])
    df['Embarked_encoded'] = le_embarked.fit_transform(df['Embarked'])

    # Выбор фичей для модели
    features = ['Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_encoded']
    X = df[features]
    y = df['Survived']

    return X, y

# Предобработка и разделение данных
print("🔄 Обрабатываем данные...")
X, y = preprocess_data(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
print("🤖 Обучаем Random Forest модель...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Сохранение модели
with open('titanic_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Оценка модели
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("=" * 50)
print("✅ Модель Titanic обучена и сохранена!")
print(f"📊 Accuracy на тренировочных данных: {train_score:.3f}")
print(f"📊 Accuracy на тестовых данных: {test_score:.3f}")
print("🎯 Используемые фичи: Pclass, Sex, Age, SibSp, Parch, Fare, Embarked")
print("=" * 50)

Overwriting train_titanic_model.py


In [41]:
# 📁 ОБНОВЛЕННАЯ ВЕРСИЯ titanic_app.py С УПРОЩЕННОЙ ФОРМОЙ
%%writefile titanic_app.py
from flask import Flask, request, jsonify, render_template_string
import pickle
import numpy as np
import os
import sys

app = Flask(__name__)

# Загрузка модели
def load_model():
    try:
        with open('titanic_model.pkl', 'rb') as f:
            model = pickle.load(f)
        print("✅ ML-модель загружена успешно!")
        return model
    except Exception as e:
        print(f"❌ Ошибка загрузки модели: {e}")
        return None

model = load_model()

# УПРОЩЕННАЯ HTML ФОРМА (как в локальной версии)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>🚢 Titanic Survival Predictor</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 500px;
            margin: 50px auto;
            padding: 20px;
            background: #f5f5f5;
        }
        .container {
            background: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .form-group { margin: 20px 0; }
        label { display: block; margin-bottom: 8px; font-weight: bold; }
        input, select {
            width: 100%;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
            font-size: 16px;
        }
        button {
            background: #007cba;
            color: white;
            padding: 15px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 18px;
            width: 100%;
            margin-top: 20px;
        }
        button:hover { background: #005a87; }
        .result {
            margin-top: 20px;
            padding: 15px;
            border-radius: 5px;
            text-align: center;
        }
        .survived { background: #d4edda; color: #155724; border: 1px solid #c3e6cb; }
        .not-survived { background: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }
        h1 { text-align: center; color: #333; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🚢 Titanic Survival Predictor</h1>

        <form method="POST" action="/predict">
            <div class="form-group">
                <label>Класс билета:</label>
                <select name="pclass" required>
                    <option value="1">1-й класс</option>
                    <option value="2">2-й класс</option>
                    <option value="3">3-й класс</option>
                </select>
            </div>

            <div class="form-group">
                <label>Пол:</label>
                <select name="sex" required>
                    <option value="male">Мужской</option>
                    <option value="female">Женский</option>
                </select>
            </div>

            <div class="form-group">
                <label>Возраст:</label>
                <input type="number" name="age" min="0" max="100" required>
            </div>

            <button type="submit">Предсказать выживание</button>
        </form>

        {% if result %}
        <div class="result {% if result.prediction == 1 %}survived{% else %}not-survived{% endif %}">
            <h3>Результат предсказания:</h3>
            <p><strong>Вероятность выживания:</strong> {{ result.probability }}%</p>
            <p><strong>Прогноз:</strong> {{ "✅ ВЫЖИВ" if result.prediction == 1 else "❌ НЕ ВЫЖИВ" }}</p>
        </div>
        {% endif %}
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if model is None:
            return "Модель не загружена", 500

        # Получение данных из формы (только 3 поля)
        pclass = int(request.form['pclass'])
        sex = request.form['sex']
        age = float(request.form['age'])

        # Автоматически заполняем остальные поля значениями по умолчанию
        sibsp = 0        # По умолчанию: нет братьев/сестер/супругов
        parch = 0        # По умолчанию: нет родителей/детей
        fare = 50.0      # По умолчанию: средняя стоимость билета
        embarked = 'S'   # По умолчанию: Саутгемптон (самый частый порт)

        # Кодирование категориальных переменных
        sex_encoded = 1 if sex == 'female' else 0
        embarked_encoding = {'C': 0, 'Q': 1, 'S': 2}
        embarked_encoded = embarked_encoding.get(embarked, 2)

        # Подготовка фичей для модели (все 7 признаков)
        features = np.array([[pclass, sex_encoded, age, sibsp, parch, fare, embarked_encoded]])

        # Предсказание
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0][1] * 100

        result = {
            'prediction': int(prediction),
            'probability': f"{probability:.1f}"
        }

        return render_template_string(HTML_TEMPLATE, result=result)

    except Exception as e:
        return f"Ошибка: {str(e)}", 500

if __name__ == '__main__':
    print("🚀 Запуск Titanic ML Service...")
    app.run(host='0.0.0.0', port=5000, debug=False)

Overwriting titanic_app.py


In [42]:
# 📁 СОЗДАЕМ ТЕСТОВЫЙ КЛИЕНТ
%%writefile test_titanic_api.py
import requests
import json
import time

def test_titanic_api():
    base_url = "http://localhost:5000"

    print("🧪 Начинаем тестирование Titanic ML Service...")
    print("=" * 50)

    # Тест 1: Проверка здоровья сервиса
    print("1. Проверка здоровья сервиса...")
    try:
        response = requests.get(f"{base_url}/health", timeout=5)
        if response.status_code == 200:
            health_data = response.json()
            print(f"   ✅ Health check: {health_data}")
        else:
            print(f"   ❌ Health check failed: {response.status_code}")
    except Exception as e:
        print(f"   ❌ Ошибка health check: {e}")

    # Тест 2: Главная страница
    print("\n2. Проверка главной страницы...")
    try:
        response = requests.get(f"{base_url}/", timeout=5)
        print(f"   ✅ Главная страница: статус {response.status_code}")
    except Exception as e:
        print(f"   ❌ Ошибка главной страницы: {e}")

    # Тест 3: API предсказания
    print("\n3. Тестирование API предсказания...")
    api_url = f"{base_url}/api/predict"

    # Пример 1: Молодая женщина, 1-й класс (высокий шанс выживания)
    test_data_1 = {
        'features': [1, 1, 25.0, 0, 0, 100.0, 0]  # 1-й класс, женский, 25 лет
    }

    try:
        response = requests.post(api_url, json=test_data_1, timeout=5)
        if response.status_code == 200:
            result = response.json()
            survival = "ВЫЖИЛА" if result['prediction'] == 1 else "НЕ ВЫЖИЛА"
            print(f"   ✅ Тест 1 (женщина 1-й класс): {survival}")
            print(f"   📊 Вероятность выживания: {result['survival_chance']}")
        else:
            print(f"   ❌ API ошибка: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"   ❌ Ошибка API запроса: {e}")

    # Пример 2: Мужчина, 3-й класс (низкий шанс выживания)
    test_data_2 = {
        'features': [3, 0, 35.0, 0, 0, 10.0, 2]  # 3-й класс, мужской, 35 лет
    }

    try:
        response = requests.post(api_url, json=test_data_2, timeout=5)
        if response.status_code == 200:
            result = response.json()
            survival = "ВЫЖИЛ" if result['prediction'] == 1 else "НЕ ВЫЖИЛ"
            print(f"   ✅ Тест 2 (мужчина 3-й класс): {survival}")
            print(f"   📊 Вероятность выживания: {result['survival_chance']}")
        else:
            print(f"   ❌ API ошибка: {response.status_code}")
    except Exception as e:
        print(f"   ❌ Ошибка API запроса: {e}")

    print("=" * 50)
    print("🎉 Тестирование завершено!")

if __name__ == "__main__":
    # Даем сервису время на запуск
    print("⏳ Ожидаем запуск сервиса...")
    time.sleep(3)
    test_titanic_api()

Overwriting test_titanic_api.py


In [43]:
# 📋 ПРОВЕРЯЕМ СОЗДАННЫЕ ФАЙЛЫ
print("📁 Проверяем созданные файлы:")
!ls -la

print("\n📄 Содержимое файлов:")
!head -5 train_titanic_model.py
!head -5 titanic_app.py

📁 Проверяем созданные файлы:
total 203672
drwxr-xr-x 1 root root     4096 Nov  5 22:10 .
drwxr-xr-x 1 root root     4096 Nov  5 21:23 ..
-rwxr-xr-x 1 root root 41210124 Oct 30 18:46 cloudflared-linux-amd64
-rw-r--r-- 1 root root 41210124 Oct 30 18:46 cloudflared-linux-amd64.1
-rw-r--r-- 1 root root 41210124 Oct 30 18:46 cloudflared-linux-amd64.2
-rw-r--r-- 1 root root 41210124 Oct 30 18:46 cloudflared-linux-amd64.3
-rw-r--r-- 1 root root 41210124 Oct 30 18:46 cloudflared-linux-amd64.4
drwxr-xr-x 4 root root     4096 Nov  4 14:36 .config
-rw-r--r-- 1 root root     2456 Nov  5 22:08 flask.log
-rw-r--r-- 1 root root       95 Nov  5 21:26 requirements.txt
drwxr-xr-x 1 root root     4096 Nov  4 14:36 sample_data
-rw-r--r-- 1 root root     3271 Nov  5 22:11 test_titanic_api.py
-rw-r--r-- 1 root root     5768 Nov  5 22:11 titanic_app.py
-rw-r--r-- 1 root root  2440813 Nov  5 21:35 titanic_model.pkl
-rw-r--r-- 1 root root     2388 Nov  5 22:11 train_titanic_model.py
-rw-r--r-- 1 root root     

In [44]:
# 📦 УСТАНАВЛИВАЕМ ВСЕ НЕОБХОДИМЫЕ БИБЛИОТЕКИ
print("📦 Устанавливаем зависимости...")

!pip install flask scikit-learn pandas numpy requests

print("✅ Зависимости установлены!")

# Проверяем установку
!pip list | grep -E "(flask|scikit|pandas|numpy|requests)"

📦 Устанавливаем зависимости...
✅ Зависимости установлены!
geopandas                                1.1.1
numpy                                    2.0.2
pandas                                   2.2.2
pandas-datareader                        0.10.0
pandas-gbq                               0.29.2
pandas-stubs                             2.2.2.240909
requests                                 2.32.4
requests-oauthlib                        2.0.0
requests-toolbelt                        1.0.0
scikit-image                             0.25.2
scikit-learn                             1.6.1
sklearn-pandas                           2.2.0


In [45]:
# 🎯 ОБУЧАЕМ МОДЕЛЬ TITANIC
print("🎯 Запускаем обучение модели...")

!python train_titanic_model.py

# Проверяем, что модель создалась
print("\n📁 Проверяем созданную модель:")
!ls -la *.pkl

print("✅ Модель готова к использованию!")

🎯 Запускаем обучение модели...
🚀 Начинаем обучение модели Titanic...
🔄 Обрабатываем данные...
/content/train_titanic_model.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/content/train_titanic_model.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [47]:
# 🚀 ЗАПУСКАЕМ FLASK СЕРВИС В ФОНОВОМ РЕЖИМЕ
import threading
import subprocess
import time
import requests

def run_flask():
    """Запускает Flask сервис"""
    print("🔄 Запускаем Flask сервис...")
    try:
        subprocess.run(["python", "titanic_app.py"], check=True)
    except Exception as e:
        print(f"")

# Запускаем в отдельном потоке
print("🔄 Подготавливаем запуск сервиса...")
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Даем время на запуск
print("⏳ Ожидаем запуск сервиса (8 секунд)...")
time.sleep(8)

# Проверяем
try:
    response = requests.get('http://localhost:5000/health', timeout=5)
    if response.status_code == 200:
        print(f"✅ Сервис успешно запущен! {response.json()}")
    else:
        print(f"⚠️  Сервис ответил с кодом: {response.status_code}")
except Exception as e:
    print(f"⚠️  Не удалось подключиться к сервису: {e}")
    print("💡 Попробуем продолжить...")

🔄 Подготавливаем запуск сервиса...
🔄 Запускаем Flask сервис...
⏳ Ожидаем запуск сервиса (8 секунд)...

✅ Сервис успешно запущен! {'model_loaded': True, 'service': 'Titanic ML Prediction Service', 'status': 'healthy'}


In [48]:
# 🧪 ТЕСТИРУЕМ РАБОТУ СЕРВИСА
print("🧪 Запускаем тесты...")

!python test_titanic_api.py

print("✅ Тестирование завершено!")

🧪 Запускаем тесты...
⏳ Ожидаем запуск сервиса...
🧪 Начинаем тестирование Titanic ML Service...
1. Проверка здоровья сервиса...
   ✅ Health check: {'model_loaded': True, 'service': 'Titanic ML Prediction Service', 'status': 'healthy'}

2. Проверка главной страницы...
   ✅ Главная страница: статус 200

3. Тестирование API предсказания...
   ✅ Тест 1 (женщина 1-й класс): ВЫЖИЛА
   📊 Вероятность выживания: 58.0%
   ✅ Тест 2 (мужчина 3-й класс): НЕ ВЫЖИЛ
   📊 Вероятность выживания: 46.0%
🎉 Тестирование завершено!
✅ Тестирование завершено!


In [51]:
# 🚀 ПРОСТОЙ ЗАПУСК ВСЕГО СРАЗУ
print("🚀 ЗАПУСКАЕМ ВЕСЬ СЕРВИС ОДНОЙ КОМАНДОЙ")

# Останавливаем все предыдущие процессы
!pkill -f "python.*titanic_app.py" || true
!pkill -f ssh || true

# Запускаем Flask в фоне
!nohup python titanic_app.py > flask.log 2>&1 &

# Ждем запуска
import time
time.sleep(8)

# Проверяем логи Flask
print("📋 Логи Flask:")
!tail -10 flask.log

# Запускаем Serveo
print("\n🌍 ЗАПУСКАЕМ SERVEO ТУННЕЛЬ...")
print("🔗 Ваша ссылка появится ниже:")
!timeout 30 ssh -o StrictHostKeyChecking=no -R 80:localhost:5000 serveo.net

🚀 ЗАПУСКАЕМ ВЕСЬ СЕРВИС ОДНОЙ КОМАНДОЙ
^C
^C
📋 Логи Flask:
✅ ML-модель загружена успешно!
🚀 Запуск Titanic ML Service...
 * Serving Flask app 'titanic_app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit

🌍 ЗАПУСКАЕМ SERVEO ТУННЕЛЬ...
🔗 Ваша ссылка появится ниже:
Forwarding HTTP traffic from https://143075a2e05cc0c06b3d5648a0fbc93e.serveo.net
HTTP request from 178.157.144.134 to https://143075a2e05cc0c06b3d5648a0fbc93e.serveo.net/
HTTP request from 178.157.144.134 to https://143075a2e05cc0c06b3d5648a0fbc93e.serveo.net/favicon.ico
HTTP request from 178.157.144.134 to https://143075a2e05cc0c06b3d5648a0fbc93e.serveo.net/predict


In [50]:
# 🌍 ПУБЛИКАЦИЯ ЧЕРЕЗ CLOUDFLARED
print("🌍 ПУБЛИКАЦИЯ ЧЕРЕЗ CLOUDFLARE TUNNEL")
print("=" * 60)
print("🚀 Устанавливаем cloudflared...")

# Скачиваем cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# Запускаем туннель
print("🔗 Запускаем туннель...")
print("📢 Ваша ссылка появится ниже!")
print("⏳ Ожидайте 15-20 секунд...")
print("=" * 60)

!./cloudflared-linux-amd64 tunnel --url http://localhost:5000

🌍 ПУБЛИКАЦИЯ ЧЕРЕЗ CLOUDFLARE TUNNEL
🚀 Устанавливаем cloudflared...
🔗 Запускаем туннель...
📢 Ваша ссылка появится ниже!
⏳ Ожидайте 15-20 секунд...
2025-11-05T22:13:40Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-05T22:13:40Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-05T22:13:45Z INF +--------------------------------------------------------------------------------------------+
2025-11-05T22:13:45Z INF |  Your quick Tunnel has been

In [28]:
pip install beefree

ERROR: Could not find a version that satisfies the requirement beefree (from versions: none)
ERROR: No matching distribution found for beefree
